# Установка библиотек

In [1]:
# Установка необходимых библиотек
!pip install gensim transformers torch scikit-learn nltk

# Импорт

In [2]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import gensim
from gensim.models import Word2Vec
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
def flatten_to_text(value):
    """
    Преобразует значение в текст:
    - Массивы: объединяются в строку через пробел.
    - Словари: значения объединяются в строку через пробел.
    - Другие типы: конвертируются в строку напрямую.
    - Пропущенные значения заменяются на пустую строку.
    """
    try:
        if isinstance(value, list):  # Проверка для списков
            return " ".join(map(str, value))
        elif isinstance(value, dict):  # Проверка для словарей
            return " ".join(map(str, value.values()))
        elif pd.isnull(value):  # Пропущенные значения
            return ""
        else:
            return str(value)  # Конвертация остальных типов
    except Exception as e:
        print(f"Ошибка при обработке значения: {value}, ошибка: {e}")
        return ""  # Возврат пустой строки при ошибке

def preprocess_dataframe(data, text_columns):
    """
    Преобразует указанные колонки в датафрейме:
    - Преобразует массивы, словари и другие значения в строки.
    - Объединяет текстовые поля в один столбец.
    """
    for col in text_columns:
        data[col] = data[col].apply(flatten_to_text)  # Преобразование каждого значения

    # Создаем общий текстовый столбец
    data['text'] = data[text_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    return data

In [4]:
import re

def clean_text(text):
    """
    Простая функция для препроцессинга текста:
    - Приводит текст к нижнему регистру.
    - Удаляет пунктуацию, слэши, и прочий "мусор".
    - Оставляет только буквы и пробелы.

    Args:
        text (str): Исходный текст.

    Returns:
        str: Очищенный текст.
    """
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление нежелательных символов (оставляем только буквы и пробелы)
    text = re.sub(r'[^а-яА-ЯёЁa-zA-Z\s]', '', text)
    # Замена множественных пробелов на один
    text = re.sub(r'\s+', ' ', text).strip()
    return text


# 1. Загрузка и обработка данных

In [ ]:
file_path = '/content/drive/MyDrive/DocsNew1/epic_records_with_target.parquet'
data = pd.read_parquet(file_path)
use_columns = ['gender', 'amnez_life', 'amnez_disease', 'condition',
       'ward_table', 'full_wards', 'complication_of_main_disease_desc',
       'main_disease_desc', 'secondary_disease_desc']

In [ ]:
data = preprocess_dataframe(data, use_columns)

# 2. Лемматизация и стемминг

In [ ]:
nltk.data.path.append("/root/nltk_data")

In [ ]:
stop_words = set(stopwords.words('russian'))
stemmer = SnowballStemmer("russian")

def preprocess_text(text):
    tokens = word_tokenize(text, language="russian")
    tokens = [stemmer.stem(word) for word in tokens if word.isalpha() and word not in stop_words]
    return " ".join(tokens)

# Применяем препроцессинг (данные для TF-IDF и модели без embeddings можно оставить с обработкой)
data['processed_text'] = data['text'].apply(preprocess_text)

In [ ]:
data['processed_text_simple'] = data['text'].apply(clean_text)

In [ ]:
data.to_parquet('/content/drive/MyDrive/DocsNew1/preprocessed_records.parquet')

# 3. Обучение собственного word embedding на данных

In [ ]:
# Токенизация текстов для обучения word2vec
tokenized_texts = [word_tokenize(text, language="russian") for text in data['text']]

In [ ]:
# Инициализация и обучение Word2Vec
w2v_model = Word2Vec(
    sentences=tokenized_texts,  # Тексты в виде токенов
    vector_size=100,            # Размер вектора embeddings
    window=5,                   # Окно для соседей
    min_count=10,               # Минимальное количество вхождений слова
    workers=2,                  # Число потоков
    sg=1                        # Используем Skip-gram
)

In [ ]:
# Сохранение обученной модели
w2v_model_path = "/content/drive/MyDrive/DocsNew1/word2vec_custom.model"
w2v_model.save(w2v_model_path)
print("Обученная модель word embeddings сохранена по пути:", w2v_model_path)

Обученная модель word embeddings сохранена по пути: /content/drive/MyDrive/DocsNew1/word2vec_custom.model


In [ ]:
# Пример использования собственного embeddings
print("Пример embedding для слова 'инфаркт':", w2v_model.wv['инфаркт'])

Пример embedding для слова 'инфаркт': [ 0.29276365  0.6517586   0.5328845   0.73116666 -0.8496222  -0.636645
  0.6585501   1.1890719  -0.79563886 -0.2130396  -0.42085025  0.15055671
  0.07983494  0.31640658 -0.5890132   0.2721545   0.14484827 -0.5062335
 -0.23533274 -0.57325673  0.2820768   0.44494748  0.845124   -0.20620432
 -0.6613513  -0.32934397  0.24691     0.38782993 -0.50931334  0.29559615
  0.84071743 -0.28958678 -0.27046475 -0.52372414  0.05379103  0.6286116
  0.65153503 -0.80114216 -0.08808368 -1.1112344   1.2720308  -0.12672056
 -0.36405873  0.21855429  0.5761397   0.9261662  -0.20644742  0.27033344
 -0.30193034  0.8543009   0.504435   -0.8382857   0.00977375 -0.65603644
 -0.44226336 -0.00469937  0.7426868   0.22981463 -0.01551979  0.7230486
 -0.15901366 -1.0127788   0.9339757  -0.8905167  -0.7562497  -0.4298161
  0.26092336  0.733011   -0.59222364  0.96409667  0.38321874 -0.4533126
  1.1089891   0.48428282  0.13257563 -0.16951585  0.2496799  -0.5567076
 -0.01357115 -0.28204

# 4. Загрузка предобученных word embeddings (например, FastText или Word2Vec)

In [ ]:
from gensim.models import KeyedVectors

# Укажите путь к предобученной модели word embeddings
embedding_model_path = "/content/drive/MyDrive/DocsNew1/word2vec_custom.model"
try:
    word_vectors = KeyedVectors.load(embedding_model_path)
    print("Предобученные word embeddings успешно загружены!")
except FileNotFoundError:
    print("Ошибка загрузки предобученной модели! Проверьте путь.")

Предобученные word embeddings успешно загружены!


# 5. Преобразование текста с использованием TF-IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=500)
tfidf_features = tfidf_vectorizer.fit_transform(data['processed_text']).toarray()

# Разделение на обучающую и валидационную выборки
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['processed_text'], data['target'], test_size=0.2, random_state=42
)

# 6. Использование word embeddings и модели трансформера

In [23]:
data = pd.read_parquet('/content/drive/MyDrive/DocsNew1/preprocessed_records.parquet')

In [24]:
# Разделение на обучающую и валидационную выборки
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['processed_text_simple'], data['target'], test_size=0.2, random_state=42
)

In [25]:
# Инициализация токенизатора и модели трансформера
model_name = 'alexyalunin/RuBioBERT'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at alexyalunin/RuBioBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
for param in model.bert.parameters():
    param.requires_grad = False
#Определение класса датасета
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [27]:
# Создание датасетов
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
val_dataset = TextDataset(val_texts, val_labels, tokenizer)

# 7. Параметры обучения

In [28]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=50,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=512,
    learning_rate=1e-3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=13,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, predictions)
    roc_auc = roc_auc_score(labels, logits[:, 1])
    return {'accuracy': acc, 'roc_auc': roc_auc}

# 8. Тренировка модели трансформера

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Roc Auc
1,0.678600,0.681775,0.547739,0.628571
2,0.658700,0.668000,0.613065,0.624417
3,0.644900,0.665685,0.613065,0.628166
4,0.630800,0.663865,0.613065,0.631307
5,0.624800,0.664629,0.613065,0.638804
6,0.620600,0.662817,0.603015,0.644681
7,0.622100,0.670263,0.623116,0.647720
8,0.620300,0.666841,0.608040,0.650051
9,0.624700,0.662564,0.608040,0.650963
10,0.613400,0.659566,0.618090,0.655117


Epoch,Training Loss,Validation Loss,Accuracy,Roc Auc
1,0.678600,0.681775,0.547739,0.628571
2,0.658700,0.668000,0.613065,0.624417
3,0.644900,0.665685,0.613065,0.628166
4,0.630800,0.663865,0.613065,0.631307
5,0.624800,0.664629,0.613065,0.638804
6,0.620600,0.662817,0.603015,0.644681
7,0.622100,0.670263,0.623116,0.647720
8,0.620300,0.666841,0.608040,0.650051
9,0.624700,0.662564,0.608040,0.650963
10,0.613400,0.659566,0.618090,0.655117


TrainOutput(global_step=650, training_loss=0.6091061012561505, metrics={'train_runtime': 2921.1399, 'train_samples_per_second': 13.556, 'train_steps_per_second': 0.223, 'total_flos': 1.0419197792256e+16, 'train_loss': 0.6091061012561505, 'epoch': 50.0})

In [32]:
# Сохранение обученной модели
model.save_pretrained('/content/drive/MyDrive/DocsNew1/rubioBERT_finetuned')
tokenizer.save_pretrained('/content/drive/MyDrive/DocsNew1/rubioBERT_finetuned_tokenizer')

('/content/drive/MyDrive/DocsNew1/rubioBERT_finetuned_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/DocsNew1/rubioBERT_finetuned_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/DocsNew1/rubioBERT_finetuned_tokenizer/vocab.txt',
 '/content/drive/MyDrive/DocsNew1/rubioBERT_finetuned_tokenizer/added_tokens.json')